In [9]:
import os
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, models


from utils.dataset import SkinDataset
from utils.utils import train, validate, test, load_data_file
from utils.metric import MetricsMonitor

## Data Transformation

In [2]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images horizontally with 50% probability
    transforms.RandomAffine(
        degrees=15,                           # Random rotation within [-15, 15] degrees
        translate=(0.1, 0.1),                 # Random shift by up to 10% in both x and y directions
    ),
    transforms.Resize((224, 224)),           # Resize to match ResNet input size
    transforms.ToTensor(),                   # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

## Load Data

In [3]:
CLASSES = ['nevus', 'others']
BATCH_SIZE = 64
EPOCHS = 100
LR = 0.0005
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [7]:
## Load data paths and labels
train_path, train_labels = load_data_file('datasets/train.txt')
train_path, val_path, train_labels, val_labels = train_test_split(train_path, train_labels, test_size=0.2, random_state=42, stratify=train_labels)
test_path, test_labels = load_data_file('datasets/val.txt')

## Create datasets and dataloaders
train_dataset = SkinDataset(train_path, train_labels, transform)
val_dataset = SkinDataset(val_path, val_labels, transform)
test_dataset = SkinDataset(test_path, test_labels, transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [8]:
print('================== Train dataset Info: ==================\n', train_dataset)
print('================== Val dataset Info: ==================\n', val_dataset)
print('================== Test dataset Info: ==================\n', test_dataset)

================== Train dataset Info: ==================
 Dataset: 12156 samples
Class distribution: {0: 6180, 1: 5976}

================== Val dataset Info: ==================
 Dataset: 3039 samples
Class distribution: {1: 1494, 0: 1545}

================== Test dataset Info: ==================
 Dataset: 3796 samples
Class distribution: {0: 1931, 1: 1865}



## Model

In [10]:
# Model
model = models.resnet50(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, len(CLASSES))
model = model.to(device)

/Users/huytrq/miniconda3/envs/py11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/huytrq/miniconda3/envs/py11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


## Training

In [11]:
# Loss and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Monitors
train_monitor = MetricsMonitor(metrics=["loss", "accuracy"])
val_monitor = MetricsMonitor(metrics=["loss", "accuracy"], patience=5, mode="max")

In [ ]:
# Training Loop
for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    train(model, train_loader, criterion, optimizer, device, train_monitor)
    validate(model, val_loader, criterion, device, val_monitor)
    val_acc = val_monitor.compute_average("accuracy")
    if val_monitor.early_stopping_check(val_acc, model):
        print("Early stopping triggered.")
        break

## Testing

In [ ]:
test(model, test_loader, device)